# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = pd.read_csv("../output_data/cities.csv" ,index_col=0)
data.reset_index(drop=True, inplace=True)
data.head()

,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Bima,79,ID,1578282998,61,-8.45,118.73,86.61,3.51
1,Bambous Virieux,75,MU,1578282772,94,-20.34,57.76,75.99,12.75
2,Jamestown,82,AU,1578283000,55,-33.21,138.60,73.40,0.31
3,Irbeyskoye,100,RU,1578283001,86,55.64,95.45,25.48,3.58
4,Lykhivka,92,UA,1578283002,93,48.69,33.92,31.57,9.28


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations 
locations = data[["Lat", "Lng"]]
#.astype(float)

data = data.dropna()
Humidity = data["Humidity"]
#.astype(float)

fig = gmaps.figure(center=(0,0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=400,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_filter = data.loc[(data["Max Temp"]>70)&(data["Max Temp"]<80)&(data["Cloudiness"]==0)&(data["Wind Speed"]<10)].dropna()
weather_filter


,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
6,Puerto Ayora,0,EC,1578283005,81,-0.74,-90.35,75.99,4.00
13,Sidrolandia,0,BR,1578283021,69,-20.93,-54.96,75.20,6.93
60,Ratnagiri,0,IN,1578283077,65,16.98,73.30,73.92,7.70
131,Concordia,0,AR,1578283174,69,-31.40,-58.02,73.99,5.50
151,San Patricio,0,PY,1578283217,53,-26.98,-56.83,78.60,8.46
198,Paita,0,PE,1578283130,71,-5.09,-81.11,75.27,5.41
201,Kyaikkami,0,MM,1578283321,66,16.08,97.57,78.87,4.38
264,Coahuayana,0,MX,1578283407,48,18.62,-100.35,72.73,3.58
415,Nova Granada,0,BR,1578283648,88,-20.53,-49.32,73.40,4.70
462,Iturama,0,BR,1578283707,88,-19.73,-50.20,70.79,6.06


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = data[["City_ID", "Country", "Lat" , "Lng"]].copy()
hotel_df["Hotel Name"]=""
hotel_df

,City_ID,Country,Lat,Lng,Hotel Name
0,Bima,ID,-8.45,118.73,
1,Bambous Virieux,MU,-20.34,57.76,
2,Jamestown,AU,-33.21,138.60,
3,Irbeyskoye,RU,55.64,95.45,
4,Lykhivka,UA,48.69,33.92,
...,...,...,...,...,...
544,Pestovo,RU,55.36,37.98,
545,Bahia Blanca,AR,-38.72,-62.27,
546,Alofi,NU,-19.06,-169.92,
547,Pinega,RU,64.70,43.39,


In [8]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params={
    "radius":5000,
    "types" : "lodging",
    "key": g_key 
}
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    response = requests.get(base_url,params=params).json()
    results = response['results']
    try:
        #print(f"{results[0]['name']}")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

    
    

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/

In [9]:
hotel_df.head()


,City_ID,Country,Lat,Lng,Hotel Name
0,Bima,ID,-8.45,118.73,Lambitu Hotel
1,Bambous Virieux,MU,-20.34,57.76,Casa Tia Villa
2,Jamestown,AU,-33.21,138.60,Jamestown Motel Terraces
3,Irbeyskoye,RU,55.64,95.45,"Gostinitsa ""Agul"""
4,Lykhivka,UA,48.69,33.92,


In [10]:
hotel_df.to_csv('../Output_Data/hotel.csv')
hotel = pd.read_csv('../Output_Data/hotel.csv')
hotel.head()

,Unnamed: 0,City_ID,Country,Lat,Lng,Hotel Name
0,0,Bima,ID,-8.45,118.73,Lambitu Hotel
1,1,Bambous Virieux,MU,-20.34,57.76,Casa Tia Villa
2,2,Jamestown,AU,-33.21,138.60,Jamestown Motel Terraces
3,3,Irbeyskoye,RU,55.64,95.45,"Gostinitsa ""Agul"""
4,4,Lykhivka,UA,48.69,33.92,NaN


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City_ID}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations1 = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
# hotel_layer = gmaps.symbol_layer(
#     locations1, fill_color='rgba(0, 150, 0, 0.4)',
#     stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
#     info_box_content = hotel_info)
hotel_layer = gmaps.marker_layer(locations1,info_box_content = hotel_info)

fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig
# Display Map



Figure(layout=FigureLayout(height='420px'))

In [13]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))